In [4]:
%matplotlib inline
import pandas as pd

import matplotlib.pylab as plt

from IPython.display import display

In [5]:
charges = pd.read_csv("../data/Charges.csv")

charges["BILL_MONTH"] = pd.to_datetime(charges["BILL_MONTH"])

charges.head()


,SUBSCRIBER_ID,BILL_MONTH,CHARGES
0,4741,2017-03-01,4.572
1,1754,2017-05-01,4.894
2,381,2017-01-01,6.670
3,5832,2016-12-01,4.299
4,5277,2017-03-01,1.610


In [6]:
changes = pd.read_csv("tariff_changes.csv")

changes["CHANGE_MONTH"] = pd.to_datetime(changes["CHANGE_MONTH"])
print("Change of plans:")
changes.head()


Change of plans:


,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,CHANGE_MONTH
0,1,2,3,2017-03-01
1,2,1,5,2017-06-01
2,3,1,5,2017-04-01
3,4,3,5,2017-01-01
4,5,1,5,2017-05-01


In [7]:
# функция разбития данных по месяцам
def add_months(col, months):
    return (col + pd.DateOffset(months=months))\
        .values.astype('datetime64[D]')


# Соединяем данные
merged = changes.merge(charges, on="SUBSCRIBER_ID")
merged["min_date"] = add_months(merged["CHANGE_MONTH"], -3)
merged["max_date"] = add_months(merged["CHANGE_MONTH"], 3)
merged = merged.query("BILL_MONTH >= min_date and BILL_MONTH <= max_date")\
    .drop(["min_date", "max_date"], axis=1)

# Помечаем данные с информацией о платежах до и после изменения ТП и сохраняем их
merged.loc[merged.eval("CHANGE_MONTH > BILL_MONTH"), "VALUE"] = "BILL_BEFORE"
merged.loc[merged.eval("CHANGE_MONTH < BILL_MONTH"), "VALUE"] = "BILL_AFTER"
merged = merged[~merged["VALUE"].isnull()]
merged.to_csv("plan_change_with_bills.csv", index=False)

merged.head()


,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,CHANGE_MONTH,BILL_MONTH,CHARGES,VALUE
0,1,2,3,2017-03-01,2016-12-01,21.132,BILL_BEFORE
2,1,2,3,2017-03-01,2017-06-01,23.651,BILL_AFTER
7,1,2,3,2017-03-01,2017-02-01,27.858,BILL_BEFORE
8,1,2,3,2017-03-01,2017-05-01,27.487,BILL_AFTER
9,1,2,3,2017-03-01,2017-04-01,23.983,BILL_AFTER


In [8]:
# Подсчитаем среднюю абонентскую плату за 3 месяца до и 3 месяца после смены тарифного плана
plan_columns = changes.columns.tolist()
mean_payments = merged.groupby(plan_columns + ["VALUE"])["CHARGES"]\
    .mean().round(2).unstack("VALUE").reset_index()

mean_payments.head()


VALUE,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,CHANGE_MONTH,BILL_AFTER,BILL_BEFORE
0,1,2,3,2017-03-01,25.04,23.09
1,2,1,5,2017-06-01,5.13,5.59
2,3,1,5,2017-04-01,5.00,5.19
3,4,3,5,2017-01-01,4.81,6.67
4,5,1,5,2017-05-01,4.79,3.72


In [9]:
# функция для подсчёта изменения абонентской платы
def payment_change(data, percent=True):
    data["PAYMENT_CHANGE"] = data.eval("BILL_AFTER - BILL_BEFORE")
    if percent:
        data["PAYMENT_CHANGE"] = data.eval("100 * PAYMENT_CHANGE / BILL_AFTER")

    pvt_payment_change = pd.pivot_table(data=data,
                                        columns="TARIFF_PLAN_ID_TO", index="TARIFF_PLAN_ID_FROM",
                                        values="PAYMENT_CHANGE", aggfunc=sum)\
        .fillna(0).round().astype(int)

    title = "Average change in payment per month (%)"
    print("%s:" % title)
    display(pvt_payment_change)
    plt.show()


tmp = mean_payments.groupby(["TARIFF_PLAN_ID_FROM", "TARIFF_PLAN_ID_TO"])[
    ["BILL_AFTER", "BILL_BEFORE"]].mean().round(2).reset_index()

payment_change(tmp, percent=True)


Average change in payment per month (%):


TARIFF_PLAN_ID_TO,1,2,3,4,5
TARIFF_PLAN_ID_FROM,,,,,
1,0,-29,-13,-40,-30
2,-14,0,-35,-43,-25
3,-12,23,0,-42,-31
4,11,13,-4,0,-3
5,-10,27,-1,-48,0


                        Conclusions:

In most cases, after a plan change, the average monthly bill of the client decreases. 

The most profitable tariff plan for change 4.

The most unprofitable plan for change 2.